In [1]:
%load_ext rpy2.ipython

In [4]:
%%R
require(phyloseq)

Loading required package: phyloseq


In [2]:
%%R
  require(reshape)
  require(picante)
  require(ape)
  require(gtools)

Loading required package: reshape
Loading required package: picante
Loading required package: ape
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.3-0
Loading required package: nlme
Loading required package: gtools

Attaching package: ‘gtools’

The following object is masked from ‘package:permute’:

    permute



In [5]:
%%R
physeq = import_biom("../../SeqData/otu_table.tax.meta.biom", "../../SeqData/trees/fulltree.tre", parseFunction = parse_taxonomy_greengenes)

In [6]:
%%R
physeq = prune_samples(sample_data(physeq)$Month==2.5, physeq)
physeq = prune_samples(sample_data(physeq)$Trtmt=="F"|sample_data(physeq)$Trtmt=="Q", physeq)
physeq = prune_samples(sample_data(physeq)$Cosm==42, physeq)
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 9190 taxa and 4 samples ]
sample_data() Sample Data:       [ 4 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 9190 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 9190 tips and 9188 internal nodes ]


In [35]:
%%R
phy=phy_tree(physeq)
samp=t(otu_table(physeq))
# Need OTU table to have samples = rows

In [37]:
%%R
 samp <- t(samp)
# flips the OTU table back so now taxa are rows
  samp <- samp[mixedsort(rownames(samp)),]
# Gets the rownames
  sums <- subset(rowSums(samp), rowSums(samp) > 0)
# Cuts out any taxa with no representatives
  samp <- samp[names(sums),]
# Redefines the OTU table to it's only those samples

OTU Table:          [3297 taxa and 4 samples]
                     taxa are rows
             67.2  39   67  24
1_197           3  39    7   1
1_232          64 135   71  33
1_382          12  32   22   2
1_601          70 278   82  44
1_713           2   4    3   0
1_1649          0   2    0   0
1_1999          8  18    4   2
1_2319          4  48    0   1
1_2703          9   4   14   0
1_2962         13  18    5   4
1_4318         19  14   21   0
1_4779          3  48    8   1
1_4949          5   2    2   0
1_5693          6 223   10  16
1_6416         55  93   64   2
1_6693          8   3    2   0
1_6803          0   4    0   0
1_9266          1   3    1   0
1_9969          3   3   11   0
1_10014         2   2    2   0
1_11013         2   2    0   0
1_11967         1   2    0   0
1_12933         0   0    8   2
1_14535         5   0    3   0
1_14853         0   3    0   0
1_17323         0   0    4   0
1_19010         3   7    3   0
1_19126         1   0    1   0
1_21395         0   

In [39]:
%%R
phy2 = prune.sample(t(samp), phy)
# Cuts out any not included taxa from the phylogenetic tree as well
phy2


Phylogenetic tree with 3297 tips and 3295 internal nodes.

Tip labels:
	69_2370, 67.2_21506, 67_26460, 96_50315, 58_38953, 73_37380, ...
Node labels:
	, 0.826, 0.898, 0.938, 0.762, 0.921, ...

Unrooted; includes branch lengths.


In [40]:
%%R
samples = length(samp[1,])
taxa = length(samp[,1])
# gets the number of samples and OTUs we have

In [47]:
%%R
dim(phy$edge)

[1] 18377     2


In [48]:
%%R
    phy$tip.label <- rep(1:length(phy$tip.label))
# Relabels the tree's OTUs to just numbers
    edge <- phy$edge[,2]
# Gets the beginning or ending node number for either tips or nodes..
  Bnode = taxa + 1
# We want one more node than we have taxa (maybe affected by whether tree is rooted or not?)
# I see at OTU 9191/9192 it jumps... not sure why...
  Li = t(t(dist.nodes(phy)[1:taxa, Bnode]))
# get the distance between all nodes from our tree
  Lb <- cbind(phy$edge[,2], phy$edge.length)
# Makes a table of the edge numbers and the edge lengths from our tree
  Lb <- Lb[order(Lb[,1]),] 
# Just orders that.
Lb

          [,1]    [,2]
    [1,]     1 0.20307
    [2,]     2 0.03937
    [3,]     3 0.00015
    [4,]     4 0.28239
    [5,]     5 0.00014
    [6,]     6 0.02090
    [7,]     7 0.01504
    [8,]     8 0.05225
    [9,]     9 0.06916
   [10,]    10 0.15197
   [11,]    11 0.08877
   [12,]    12 0.15353
   [13,]    13 0.22568
   [14,]    14 0.02416
   [15,]    15 0.00669
   [16,]    16 0.03371
   [17,]    17 0.01159
   [18,]    18 0.08176
   [19,]    19 0.00927
   [20,]    20 0.03057
   [21,]    21 0.14434
   [22,]    22 0.12682
   [23,]    23 0.07583
   [24,]    24 0.01416
   [25,]    25 0.02205
   [26,]    26 0.01789
   [27,]    27 0.00821
   [28,]    28 0.01929
   [29,]    29 0.02884
   [30,]    30 0.03558
   [31,]    31 0.01084
   [32,]    32 0.01262
   [33,]    33 0.01718
   [34,]    34 0.02337
   [35,]    35 0.01086
   [36,]    36 0.11254
   [37,]    37 0.02084
   [38,]    38 0.01668
   [39,]    39 0.01657
   [40,]    40 0.19290
   [41,]    41 0.04557
   [42,]    42 0.03756
   [43,]   

In [50]:
%%R 
  p=matrix(0,taxa,samples)
#creates empty matrix of correct dimensions (taxa (rows) x  samples (cols))

In [51]:
%%R
  for (k in 1:samples){
    p[,k]<-samp[,k]/sum(samp[,k])
  }
# Populates the matrix with the relative abundance values for each OTU

                [,1]         [,2]         [,3]         [,4]
   [1,] 1.221250e-04 1.004947e-03 2.364465e-04 0.0001905125
   [2,] 2.605333e-03 3.478664e-03 2.398244e-03 0.0062869118
   [3,] 4.884999e-04 8.245723e-04 7.431177e-04 0.0003810250
   [4,] 2.849583e-03 7.163471e-03 2.769802e-03 0.0083825491
   [5,] 8.141665e-05 1.030715e-04 1.013342e-04 0.0000000000
   [6,] 0.000000e+00 5.153577e-05 0.000000e+00 0.0000000000
   [7,] 3.256666e-04 4.638219e-04 1.351123e-04 0.0003810250
   [8,] 1.628333e-04 1.236858e-03 0.000000e+00 0.0001905125
   [9,] 3.663749e-04 1.030715e-04 4.728931e-04 0.0000000000
  [10,] 5.292082e-04 4.638219e-04 1.688904e-04 0.0007620499
  [11,] 7.734582e-04 3.607504e-04 7.093396e-04 0.0000000000
  [12,] 1.221250e-04 1.236858e-03 2.702246e-04 0.0001905125
  [13,] 2.035416e-04 5.153577e-05 6.755616e-05 0.0000000000
  [14,] 2.442499e-04 5.746238e-03 3.377808e-04 0.0030481997
  [15,] 2.238958e-03 2.396413e-03 2.161797e-03 0.0003810250
  [16,] 3.256666e-04 7.730365e-05 6.7556

In [ ]:
%%R
  That=matrix(0,1,samples)
  for (k in 1:samples){
    That[,k] <-sum(Li * p[,k])
  }
# Creates a matrix, awkwardly named "That", which fills itself 

In [ ]:
%%R


##########    This function returns the descendents of any internal node   #################################
  internal2tips.self = function (phy, int.node){
    #require(picante); require(ape)
    Ntaxa = length(phy$tip.label)
    Nnode = phy$Nnode
    if ((Ntaxa + Nnode - 1) != nrow(phy$edge)) {
      print("tree structure error")
      break
    }
    if (mode(int.node) == "character"){
      nodes = which(phy$node.label == int.node) + Ntaxa
    }else nodes = int.node
    tips = c()
    repeat {
      nodes = phy$edge[which(phy$edge[, 1] %in% nodes), 2]
      if (length(nodes) == 0)
        break
      tips = c(tips, nodes)
    }
    tips = tips[tips <= Ntaxa]
    if( int.node <= Ntaxa & length(tips) == 0 ){
      tips = int.node
    }
    tips = phy$tip.label[tips]
    return(tips)
  }
  
############################################################################################################
#####This builds the Z matrix from your phylogeny ######################################################
  ########################################################
  
  S <- rep(1:length(phy$tip.label))
  sp <- matrix(NA, nrow = length(S) + 1, ncol = length(edge) + 1)
  sp[1,-1] <- sort(edge)
  sp[-1,1] <- S
  edges <- sp[1,]
  
  for (i in (2:ncol(sp))) {
    branch <- internal2tips.self(phy,edges[i])
    
    for (j in (2:nrow(sp))){
      sp[j,i] <- ifelse(sp[j,1] %in% branch, 1, 0)
    }
  }
  
  hs <- sp[-1,-1]
  colnames(hs) <-sort(edge)
  rownames(hs) <- S
  hs <- melt(hs, varnames = c("species", "branch"))
  for (k in 1:samples){
    hs[,k + 3] <- rep(p[,k], length(edge))
  }

  hs$Li <- rep(Li, length(edge))
  hs <- hs[order(hs$species), ]
  hs$Lb <- rep(Lb[,2], length(phy$tip.label))
  hs <- (subset(hs, hs$value != 0))
  hs <- hs[order(hs$species, hs$branch), ]
  
  
  
  Z <- matrix(NA, nrow(hs), nrow(hs))
  colnames(Z) <- hs$species
  rownames(Z) <- hs$branch
  
  for(i in 1:length(hs$branch)){
    for(j in 1:length(hs$species)){
      branch <- internal2tips.self(phy,hs$branch[i])
      
      Z[i,j] <- ifelse(hs$species[j] %in% branch, That/Li[hs$species[j]], 0)
    }
  }

pi <- matrix(0, ncol(Z), samples)
  for (k in 1:samples){
pi[,k] <- (hs$Lb/That[,k])*hs[,k+3]
}
  
  lenq <- 50
  qq <- seq(length = lenq, from  = 0, by = .11)
  
  # Initialise the Zp matrix to zero
  Zp=matrix(0,ncol(Z), samples)
  
  
  # Compute Zp
  for (k in 1:samples) {
    for (i in 1:ncol(Z)){
      for (j in 1:ncol(Z)){
        Zp[i,k]<-Zp[i,k]+Z[i,j]*pi[j,k]
      }
    }}

  
  
  # Initialise the Diversity matrix to zero
  Dqz = matrix(0, lenq ,samples)
  
  #  Loop to calculate the diversity Dqz for each value of q (iq) and each sample (k)
  
  for (k in 1:samples) {
    
    for (iq in 1:lenq)  {
      q<-qq[iq];
      
      for (zpi in 1:length(Zp[,k])){
        if (Zp[zpi,k]>0)(
          Dqz[iq,k]<-Dqz[iq,k]+ pi[zpi,k]*(Zp[zpi,k])^(q-1))
      }
      
      Dqz[iq,k] <- Dqz[iq,k]^(1/(1-q));
    }
  }
  
  return(Dqz)
}


In [7]:
%%R
#From https://gist.github.com/darmitage/2649763
############Here's a function that calculates Hill number diversity using a phylogeny from the Leinster and Cobbold (2012) paper

#Requires a phylogeny with tip labels and a community matrix (rows = samples, columns = species)
Phylo.Z <- function(phy,samp){

#packages required for this to work:
  require(reshape)
  require(picante)
  require(ape)
  require(gtools)
  
  phy <- prune.sample(samp,phy)
  samp <- t(samp)
  samp <- samp[mixedsort(rownames(samp)),]
  sums <- subset(rowSums(samp), rowSums(samp) > 0)
  samp <- samp[names(sums),]
  samples = length(samp[1,])
  taxa = length(samp[,1])
  phy$tip.label <- rep(1:length(phy$tip.label))
  edge <- phy$edge[,2]
    Bnode = taxa + 1
  Li = t(t(dist.nodes(phy)[1:taxa, Bnode]))
  Lb <- cbind(phy$edge[,2], phy$edge.length)
  Lb <- Lb[order(Lb[,1]),] 
  
  p=matrix(0,taxa,samples)
  
  for (k in 1:samples){
    p[,k]<-samp[,k]/sum(samp[,k])
  }
  
  That=matrix(0,1,samples)
  for (k in 1:samples){
    That[,k] <-sum(Li * p[,k])
  }

##########    This function returns the descendents of any internal node   #################################
  internal2tips.self = function (phy, int.node){
    #require(picante); require(ape)
    Ntaxa = length(phy$tip.label)
    Nnode = phy$Nnode
    if ((Ntaxa + Nnode - 1) != nrow(phy$edge)) {
      print("tree structure error")
      break
    }
    if (mode(int.node) == "character"){
      nodes = which(phy$node.label == int.node) + Ntaxa
    }else nodes = int.node
    tips = c()
    repeat {
      nodes = phy$edge[which(phy$edge[, 1] %in% nodes), 2]
      if (length(nodes) == 0)
        break
      tips = c(tips, nodes)
    }
    tips = tips[tips <= Ntaxa]
    if( int.node <= Ntaxa & length(tips) == 0 ){
      tips = int.node
    }
    tips = phy$tip.label[tips]
    return(tips)
  }
  
############################################################################################################
#####This builds the Z matrix from your phylogeny ######################################################
  ########################################################
  
  S <- rep(1:length(phy$tip.label))
  sp <- matrix(NA, nrow = length(S) + 1, ncol = length(edge) + 1)
  sp[1,-1] <- sort(edge)
  sp[-1,1] <- S
  edges <- sp[1,]
  
  for (i in (2:ncol(sp))) {
    branch <- internal2tips.self(phy,edges[i])
    
    for (j in (2:nrow(sp))){
      sp[j,i] <- ifelse(sp[j,1] %in% branch, 1, 0)
    }
  }
  
  hs <- sp[-1,-1]
  colnames(hs) <-sort(edge)
  rownames(hs) <- S
  hs <- melt(hs, varnames = c("species", "branch"))
  for (k in 1:samples){
    hs[,k + 3] <- rep(p[,k], length(edge))
  }

  hs$Li <- rep(Li, length(edge))
  hs <- hs[order(hs$species), ]
  hs$Lb <- rep(Lb[,2], length(phy$tip.label))
  hs <- (subset(hs, hs$value != 0))
  hs <- hs[order(hs$species, hs$branch), ]
  
  
  
  Z <- matrix(NA, nrow(hs), nrow(hs))
  colnames(Z) <- hs$species
  rownames(Z) <- hs$branch
  
  for(i in 1:length(hs$branch)){
    for(j in 1:length(hs$species)){
      branch <- internal2tips.self(phy,hs$branch[i])
      
      Z[i,j] <- ifelse(hs$species[j] %in% branch, That/Li[hs$species[j]], 0)
    }
  }

pi <- matrix(0, ncol(Z), samples)
  for (k in 1:samples){
pi[,k] <- (hs$Lb/That[,k])*hs[,k+3]
}
  
  lenq <- 50
  qq <- seq(length = lenq, from  = 0, by = .11)
  
  # Initialise the Zp matrix to zero
  Zp=matrix(0,ncol(Z), samples)
  
  
  # Compute Zp
  for (k in 1:samples) {
    for (i in 1:ncol(Z)){
      for (j in 1:ncol(Z)){
        Zp[i,k]<-Zp[i,k]+Z[i,j]*pi[j,k]
      }
    }}

  
  
  # Initialise the Diversity matrix to zero
  Dqz = matrix(0, lenq ,samples)
  
  #  Loop to calculate the diversity Dqz for each value of q (iq) and each sample (k)
  
  for (k in 1:samples) {
    
    for (iq in 1:lenq)  {
      q<-qq[iq];
      
      for (zpi in 1:length(Zp[,k])){
        if (Zp[zpi,k]>0)(
          Dqz[iq,k]<-Dqz[iq,k]+ pi[zpi,k]*(Zp[zpi,k])^(q-1))
      }
      
      Dqz[iq,k] <- Dqz[iq,k]^(1/(1-q));
    }
  }
  
  return(Dqz)
}


In [ ]:
%%R
#######################################################    DONE!!!! ##############################################
#Here's an example:

#data(phylocom)
#samp = phylocom$sample
#phy = prune.sample(samp, phylocom$phylo)

samp = OTUs
phy = Tree

the.answer <- Phylo.Z(phy = phy, samp = samp)

In [ ]:
# Plot the diversity profiles for all the samples (e.g. understorey and canopy) on the same graph.
  matplot(qq,the.answer, ylim = c(min(the.answer),max(the.answer)), xlim = c(0,5),
          xlab="Sensitivity parameter q",
          ylab="Diversity DqZ(p)",
          main="Diversity profile using a similarity matrix"
          )
 